In [65]:
import numpy as np
import pandas as pd
import json

In [66]:
with open('data_analysis.json') as f:
    data = json.load(f)

In [67]:
print 'Number of records: ' + str(len(data))

Number of records: 1000000


In [68]:
data[:3]

[{u'address': u'10085 SCRIPPS RANCH CT STE A',
  u'category_code': u'44420000',
  u'city': u'SAN DIEGO',
  u'headcount': u'50 to 99',
  u'name': u'AMD CUSTOM',
  u'phone': u'3123628000',
  u'revenue': u'$20 to 50 Million',
  u'state': u'CA',
  u'time_in_business': u'10+ years',
  u'zip': u'92131'},
 {u'address': u'2566 SHALLOWFORD RD NE STE 104 # 302',
  u'category_code': u'31490000',
  u'city': u'ATLANTA',
  u'headcount': u'1 to 4',
  u'name': u'Real Hope Real Estate Inc',
  u'phone': None,
  u'revenue': u'Less Than $500,000',
  u'state': u'GA',
  u'time_in_business': u'10+ years',
  u'zip': u'30345'},
 {u'address': u'212 E MAIN ST',
  u'category_code': u'53120000',
  u'city': u'NEOSHO',
  u'headcount': u'1 to 4',
  u'name': u'Jimmy Sexton Photography',
  u'phone': u'4046331779',
  u'revenue': u'Less Than $500,000',
  u'state': u'MO',
  u'time_in_business': u'10+ years',
  u'zip': u'64850'}]

JSON data is not nested so we can easily read it into a Pandas dataframe and take advantage of Pandas methods.

In [69]:
df = pd.read_json('data_analysis.json')

In [70]:
df.head()

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
0,10085 SCRIPPS RANCH CT STE A,44420000,SAN DIEGO,50 to 99,AMD CUSTOM,3123628000,$20 to 50 Million,CA,10+ years,92131
1,2566 SHALLOWFORD RD NE STE 104 # 302,31490000,ATLANTA,1 to 4,Real Hope Real Estate Inc,None,"Less Than $500,000",GA,10+ years,30345
2,212 E MAIN ST,53120000,NEOSHO,1 to 4,Jimmy Sexton Photography,4046331779,"Less Than $500,000",MO,10+ years,64850
3,6032 CHEROKEE DR,54000000,CINCINNATI,1 to 4,YOU'RE ART,4174513798,"Less Than $500,000",OH,10+ years,45243
4,1315 N WOOSTER AVE,54100000,STRASBURG,1 to 4,Hayberg Restoration Network LLC,5135612584,"$500,000 to $1 Million",OH,10+ years,44680


Some leading zeros get dropped when reading the json and inferring data types. This would need to be fixed if we wanted to check for proper format for some of the columns (such as zip code).

### 1. Fill rate

In [71]:
# identify values equal to None
df.isnull().sum()

address                 14
category_code           14
city                    14
headcount            37648
name                    14
phone               409111
revenue              56908
state                   14
time_in_business     83875
zip                     12
dtype: int64

In [72]:
missing = df.isnull().sum()
fillrate = 1000000 - missing
print 'Fill rate for each field:\n'
print fillrate

Fill rate for each field:

address             999986
category_code       999986
city                999986
headcount           962352
name                999986
phone               590889
revenue             943092
state               999986
time_in_business    916125
zip                 999988
dtype: int64


### 2. True-valued fill rate

In [73]:
# take a look at values for a field with limited possible values, to try to identify bad data
df.headcount.unique()

array([u'50 to 99', u'1 to 4', u'5 to 9', u'10 to 19', u'20 to 49', None,
       u'100 to 249', u'250 to 499', u'500 to 999', u'Over 1,000', u'0',
       u'none', 0, u'null', u' ', u''], dtype=object)

In [74]:
non_relevant = df.headcount.unique()[-6:]

print 'Other likely non-relevant values:'
print non_relevant

Other likely non-relevant values:
[u'0' u'none' 0 u'null' u' ' u'']


In [75]:
df1 = df.where(~df.isin(non_relevant), None)

In [77]:
print 'True-valued fill rate for each field:\n'
1000000 - df1.isnull().sum()

True-valued fill rate for each field:



address             999898
category_code       999910
city                999895
headcount           962273
name                999910
phone               590798
revenue             943001
state               999896
time_in_business    916048
zip                 999890
dtype: int64

Perhaps we should check if there are any longer blank strings...

In [78]:
def strip_values(val):
    if val:
        return val.strip()
    
df_stripped = df1.applymap(strip_values)
df_no_empties = df_stripped.where(df_stripped != '', None)

In [79]:
# results are the same
print 'True-valued fill rate for each field:\n'
1000000 - df_no_empties.isnull().sum()

True-valued fill rate for each field:



address             999898
category_code       999910
city                999895
headcount           962273
name                999910
phone               590798
revenue             943001
state               999896
time_in_business    916048
zip                 999890
dtype: int64

### 3. Cardinality

In [80]:
print 'Number of unique values for each field:'
df1.apply(lambda x: len(x.unique()))

Number of unique values for each field:


address             892115
category_code         1179
city                 13715
headcount               10
name                890718
phone               575149
revenue                 12
state                   54
time_in_business         6
zip                  26392
dtype: int64

### 4. Something else interesting...

Which zip codes have the highest number (percentage-wise) of companies in each `time_in_business` category?

In [81]:
df1.time_in_business.unique()

array([u'10+ years', u'6-10 years', None, u'1-2 years', u'3-5 years',
       u'Less than a year'], dtype=object)

In [82]:
# total number of businesses by zip
total_counts = df1.zip.value_counts()

In [83]:
# just look at the top 100 zips in terms of total count
df2 = df1[df1.zip.isin(total_counts.index[:100])]

In [84]:
# counts for each zip, for each 'time_in_business' category
counts = df2.groupby('time_in_business').zip.value_counts()

In [85]:
# get a list of total counts matching the 'counts' series, 
# to divide 'counts' by and get a percentage for each category
totals = [total_counts[x] for x in counts.index.get_level_values(1)]

In [86]:
normalized = counts/totals

# show the top 10 by percentage for each category
for i in normalized.index.get_level_values(0).unique():
    print i
    print normalized[i].nlargest(10)

1-2 years
zip
85016    0.028061
30305    0.025189
10012    0.024938
29607    0.024450
77027    0.024450
10003    0.023891
60654    0.023861
91730    0.022613
37203    0.022167
75201    0.020785
Name: zip, dtype: float64
10+ years
zip
90025    0.808858
92612    0.793814
20850    0.790826
78701    0.790490
32789    0.789894
21401    0.789030
75006    0.788918
33431    0.787234
90670    0.785340
93401    0.783333
Name: zip, dtype: float64
3-5 years
zip
29607    0.070905
30339    0.058952
60007    0.054435
10016    0.053089
78216    0.052506
78759    0.050955
94111    0.050847
80302    0.050239
94612    0.050125
10013    0.049270
Name: zip, dtype: float64
6-10 years
zip
98052    0.155131
94107    0.126168
10010    0.124777
60654    0.123644
83702    0.121588
60563    0.121519
94538    0.121348
92626    0.121277
94901    0.121140
37027    0.119923
Name: zip, dtype: float64
